# Trajectory prediction

## Setup

In [ ]:

from pathlib import Path
from tqdm import tqdm
import seaborn as sns
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import numpy as np

from src.data.download import download_ais_data, download_file
from src.data.cleaning import process_multiple_zip_files
from src.data.preprocessing import (
    load_and_prepare_data,
    create_sequences,
    split_by_vessel,
    normalize_data,
)
from src.models import TrajectoryDataset, EncoderDecoderGRU, EncoderDecoderGRUWithAttention
from src.utils.model_utils import HaversineLoss, train_model, evaluate_model, create_prediction_sequences, predict_trajectories, plot_training_history, load_model_and_config, visualize_predictions
from src.visualization import plot_trajectory_comparison, create_prediction_map, create_trajectory_map
from src.utils import set_seed, haversine_distance

sns.set_style("darkgrid")

# Constants
DATA_DIR = Path("data")
MODEL_PATH = "best_model_encoder_decoder.pt"
MODEL = EncoderDecoderGRUWithAttention
INPUT_HOURS = 2
OUTPUT_HOURS = 1
SAMPLING_RATE = 5
HIDDEN_SIZE = 256
NUM_LAYERS = 3
BATCH_SIZE = 512
EPOCHS = 100
LEARNING_RATE = 0.000001
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
SEED = 42

set_seed(SEED)

print(u"Using device: ", DEVICE)

## Data

### Acquisition

In [ ]:
ZIP_NAMES = [
    "aisdk-2024-03-01.zip",
    # "aisdk-2024-03-02.zip",
    # "aisdk-2024-03-03.zip",
    # "aisdk-2024-03-04.zip",
    # "aisdk-2024-03-05.zip",
    # "aisdk-2024-03-06.zip",
    # "aisdk-2024-03-07.zip",
    # "aisdk-2024-03-08.zip",
    # "aisdk-2024-03-09.zip",
    # "aisdk-2024-03-10.zip"
]
# ZIP_NAMES = [] // Uncomment to download all files


if len(ZIP_NAMES) == 0:
    YEAR = "2024"
    MAX_WORKERS = 8
    download_ais_data(YEAR, DATA_DIR, MAX_WORKERS)
else:
    for ZIP_NAME in ZIP_NAMES:
        download_file("http://aisdata.ais.dk/2024/" + ZIP_NAME, DATA_DIR / ZIP_NAME)

### Cleaning

In [ ]:
process_multiple_zip_files(DATA_DIR)

In [ ]:
df = load_and_prepare_data(DATA_DIR)
sequences, targets, mmsi_labels, feature_cols = create_sequences(
    df, INPUT_HOURS, OUTPUT_HOURS, SAMPLING_RATE
)

X_train, X_val, X_test, y_train, y_val, y_test = split_by_vessel(
    sequences, targets, mmsi_labels, train_ratio=0.7, val_ratio=0.15, random_seed=42
)

X_train, X_val, X_test, y_train, y_val, y_test, input_scaler, output_scaler = normalize_data(
    X_train, X_val, X_test, y_train, y_val, y_test
)

train_dataset = TrajectoryDataset(X_train, y_train)
val_dataset = TrajectoryDataset(X_val, y_val)
test_dataset = TrajectoryDataset(X_test, y_test)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=8)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=8)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=8)

input_size = len(feature_cols)
output_timesteps = y_train.shape[1] // 2

### Overview

In [ ]:
print(f"Total rows: {len(df):,}")
print(f"Unique vessels (MMSI): {df['MMSI'].n_unique()}")
df.head()

In [ ]:
MAX_VESSELS = 100
create_trajectory_map(df, MAX_VESSELS)

## Training

In [ ]:
model = MODEL(
    input_size=input_size,
    hidden_size=HIDDEN_SIZE,
    num_layers=NUM_LAYERS,
    output_seq_len=output_timesteps,
    dropout=0.3,
).to(DEVICE)
print(f"\nModel architecture:")
print(model)
print(f"\nTotal parameters: {sum(p.numel() for p in model.parameters()):,}")

In [ ]:
criterion = HaversineLoss(output_scaler).to(DEVICE)
optimizer = torch.optim.AdamW(model.parameters(), lr=LEARNING_RATE, weight_decay=1e-5)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode="min", factor=0.5, patience=10)

In [ ]:
print(f"\nStarting training for {EPOCHS} epochs...")
train_losses = []
val_losses = []
best_val_loss = float("inf")
patience_counter = 0
early_stop_patience = 20

In [ ]:

for epoch in range(EPOCHS):
    teacher_forcing_ratio = max(0.2, 1.0 - (0.8 * (epoch / EPOCHS)))
    train_loss = train_model(model, train_loader, criterion, optimizer, DEVICE, epoch, EPOCHS, teacher_forcing_ratio)
    val_loss = evaluate_model(model, val_loader, criterion, DEVICE)

    train_losses.append(train_loss)
    val_losses.append(val_loss)

    scheduler.step(val_loss)

    print(f"Epoch [{epoch+1}/{EPOCHS}] - " f"Train Loss: {train_loss:.6f}, Val Loss: {val_loss:.6f}")

    if val_loss < best_val_loss:
        best_val_loss = val_loss
        patience_counter = 0
        torch.save(
            {
                "epoch": epoch,
                "model_state_dict": model.state_dict(),
                "optimizer_state_dict": optimizer.state_dict(),
                "val_loss": val_loss,
                "input_scaler": input_scaler,
                "output_scaler": output_scaler,
                "config": {
                    "input_size": input_size,
                    "hidden_size": HIDDEN_SIZE,
                    "num_layers": NUM_LAYERS,
                    "output_seq_len": output_timesteps,
                    "input_hours": INPUT_HOURS,
                    "output_hours": OUTPUT_HOURS,
                    "sampling_rate": SAMPLING_RATE,
                    "feature_cols": feature_cols,
                },
            },
            MODEL_PATH,
        )
        print(f"  -> Saved best model (val_loss: {val_loss:.6f})")
    else:
        patience_counter += 1
        if patience_counter >= early_stop_patience:
            print(f"\nEarly stopping triggered after {epoch+1} epochs")
            break

print(f"\nTraining complete! Best validation loss: {best_val_loss:.6f}")

plot_training_history(train_losses, val_losses)

## Testing

In [ ]:
checkpoint = torch.load(MODEL_PATH, map_location=torch.device(DEVICE), weights_only=False)
model.load_state_dict(checkpoint["model_state_dict"])
test_loss = evaluate_model(model, test_loader, criterion, DEVICE)
print(f"Final Test Loss: {test_loss:.6f}")

In [ ]:
visualize_predictions(model, test_loader, output_scaler, DEVICE)

## Results

In [ ]:
N_VESSELS = 25

model, config, input_scaler, output_scaler = load_model_and_config(
    MODEL_PATH, MODEL
)

sequences, targets, mmsi_list, full_trajectories, timestamps_list = create_prediction_sequences(
    df, config, n_vessels=N_VESSELS
)

predictions, _ = predict_trajectories(model, sequences, input_scaler, output_scaler)

# plot_trajectory_comparison(
#     full_trajectories, 
#     predictions, 
#     mmsi_list, 
#     config["output_hours"], 
# )

create_prediction_map(
    full_trajectories, 
    predictions, 
    mmsi_list, 
    config["output_hours"], 
    "output.html"
)

# Statistics

In [ ]:
def pointwise_haversine(y_true, y_pred):
    """
    Compute haversine distance at each time step, for each item in batch.

    y_true, y_pred: shape (B, T, 2) with [..., 0]=lat_deg, [..., 1]=lon_deg
    Returns: distances with shape (B, T) in meters.
    """

    lat_true = y_true[..., 0]
    lon_true = y_true[..., 1]
    lat_pred = y_pred[..., 0]
    lon_pred = y_pred[..., 1]

    return haversine_distance(lat_true, lon_true, lat_pred, lon_pred)



## Mean Haversine Error

In [ ]:
def mean_haversine_error(y_true, y_pred):
    """
    Mean haversine error over all batch items and time steps.
    """
    dists = pointwise_haversine(y_true, y_pred)  # (B, T)
    return float(dists.mean())

## Root Mean Squared Error (RMSE)

In [ ]:
def rmse_haversine(y_true, y_pred):
    """
    RMSE of haversine distances in meters.
    """
    dists = pointwise_haversine(y_true, y_pred)  # (B, T)
    return float(np.sqrt(np.mean(dists ** 2))) 

## Average Displacement Error (ADE)

In [ ]:

def ade(y_true, y_pred):
    """
    ADE: Average Displacement Error over whole trajectories.
    Here it's effectively the same as mean_haversine_error
    """
    dists = pointwise_haversine(y_true, y_pred)  # (B, T)
    return float(dists.mean())

## Final Displacement Error (FDE)

In [ ]:
def fde(y_true, y_pred):
    """
    FDE: error at the final time step (averaged over batch).
    """
    dists = pointwise_haversine(y_true, y_pred)  # (B, T)
    final_dists = dists[:, -1]  # last time step
    return float(final_dists.mean())

## Dynamic Time Warping Distance (DTW)

In [ ]:
def dtw_distance_trajectory(traj_true, traj_pred):
    """
    DTW distance between two single trajectories.

    traj_true, traj_pred: shape (T, 2) with [lat_deg, lon_deg]
    Returns: DTW distance in meters (normalized by path length).
    """

    T1 = traj_true.shape[0]
    T2 = traj_pred.shape[0]

    # Cost matrix: pairwise haversine distances
    # Shape (T1, T2)
    lat1 = traj_true[:, 0][:, None]  # (T1, 1)
    lon1 = traj_true[:, 1][:, None]  # (T1, 1)
    lat2 = traj_pred[:, 0][None, :]  # (1, T2)
    lon2 = traj_pred[:, 1][None, :]  # (1, T2)

    cost = haversine_distance(lat1, lon1, lat2, lon2)

    # Accumulated cost matrix
    acc = np.zeros((T1, T2), dtype=np.float64)
    acc[0, 0] = cost[0, 0]

    # First row
    for j in range(1, T2):
        acc[0, j] = cost[0, j] + acc[0, j - 1]

    # First column
    for i in range(1, T1):
        acc[i, 0] = cost[i, 0] + acc[i - 1, 0]

    # Rest
    for i in range(1, T1):
        for j in range(1, T2):
            acc[i, j] = cost[i, j] + min(
                acc[i - 1, j],      # insertion
                acc[i, j - 1],      # deletion
                acc[i - 1, j - 1],  # match
            )

    # Option 1: raw DTW distance
    # return float(acc[-1, -1])

    # Option 2: normalized by path length (more comparable)
    path_length = T1 + T2  # rough normalization
    return float(acc[-1, -1] / path_length)


def dtw_batch_mean(y_true, y_pred):
    """
    Mean DTW distance over a batch of trajectories.

    y_true, y_pred: shape (B, T, 2)
    Returns: scalar, average DTW distance in meters.
    """

    B = y_true.shape[0]
    dtw_values = []

    for b in range(B):
        dtw_b = dtw_distance_trajectory(y_true[b], y_pred[b])
        dtw_values.append(dtw_b)

    return float(np.mean(dtw_values))


In [ ]:

y_true = np.array([
    [[55.0, 12.0], [55.001, 12.002], [55.002, 12.004], [55.003, 12.006], [55.004, 12.008]],
    [[56.0, 10.0], [56.001, 10.002], [56.002, 10.004], [56.003, 10.006], [56.004, 10.008]],
])

# Slightly perturbed predictions
y_pred = y_true.copy()
y_pred += np.random.normal(scale=0.0005, size=y_true.shape)  # small noise

print("Mean Haversine Error (km):", mean_haversine_error(y_true, y_pred))
print("Root Mean Squared Error (km):      ", rmse_haversine(y_true, y_pred))
print("Average Displacement (km):                 ", ade(y_true, y_pred))
print("Final Displacement (km):                 ", fde(y_true, y_pred))
print("Dynamic Time Warping Distance (km):            ", dtw_batch_mean(y_true, y_pred))


1. Mean Haversine Error

Overall accuracy. Shows average geo prediction error in meters.

2. RMSE

Penalizes the larger mistakes more. Helps detect if the model makes big errors.

3. ADE (Average Displacement Error)

Accuracy over entire predicted trajectory. SHows how model follows true path in general.

4. FDE (Final Displacement Error)

How far it is from final position. Sometimes error grow over time.

5. DTW (Dynamic Time Warping)

Compares shape of predicted and real traj. Used to see if model captures realistic movement patt.
